In [1]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sypar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def spimi_invert(tokenized_docs):
    index = {}
    doc_id = 0
    
    for doc in tokenized_docs:
        for term in doc:
            if term not in index:
                index[term] = []
            if doc_id not in index[term]:
                index[term].append(doc_id)
                
        doc_id += 1
    
    return index

In [8]:
documents = [
    "apple banana orange",
    "banana orange strawberry",
    "apple strawberry",
    "orange"
]

In [9]:
tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
index = spimi_invert(tokenized_docs)

In [10]:
index

{'apple': [0, 2], 'banana': [0, 1], 'orange': [0, 1, 3], 'strawberry': [1, 2]}

#### X OR Y

In [26]:
def boolean_or(query_terms):
    result = []
    for term in query_terms:
        if term in index:
            result += index[term] # term의 인덱스 모두 result 넣기
    return list(set(result))   # 중복 값 없애기

In [27]:
boolean_or(["apple", "banana"])

[0, 1, 2]

#### X AND Y

In [42]:
def boolean_and(query_terms):
    result = None
    for term in query_terms:
        if term in index:
            if result is None:
                result = set(index[term])
            else:
                # intersection_update
                # 집합(set) 객체의 메서드 중 하나
                # 집합 객체 자체를 다른 집합과의 교집합으로 업데이트
                result.intersection_update(index[term])
        else:
            return []
    return list(result)

In [43]:
boolean_and(["apple", "strawberry"])

[2]

#### X AND NOT Y

In [44]:
def boolean_and_not(query_terms):
    # 질의가 두 개의 용어로 이루어져 있지 않으면 빈 리스트 반환
    # 유효한 질의 형태 아닌 경우 처리하기 위함
    if len(query_terms) != 2:
        return []
    
    term1, term2 = query_terms
    # term1이 존재하고 term2가 존재하지 않는 문서ID들을 반환
    if term1 in index and term2 in index:
        return list(set(index[term1]) - set(index[term2]))
    # term1만 index에 존재하는 경우, term1이 존재하는 ID 반환
    elif term1 in index:
        return index[term1]
    # term1과 term2 모두 index에 존재하지 않는 경우 빈리스트 반환
    else:
        return []

In [52]:
boolean_and_not(["apple", "orange"])

[2]

#### X OR NOT Y

In [53]:
def boolean_or_not(query_terms):
    if len(query_terms) != 2:
        return []
    
    term1, term2 = query_terms
    if term1 in index and term2 in index:
        return list(set(index[term1] + list(set(range(len(documents))) - set(index[term2]))))
    elif term1 in index:
        return index[term1]
    elif term2 in index:
        return list(set(range(len(documents))) - set(index[term2]))
    else:
        return []

In [ ]:
boolean_or_not(["apple", "orange"])